# 네이버 영화 리뷰 데이터_감정분석_ ML
## 데이터 호출

In [1]:
import pandas as pd

df = pd.DataFrame(pd.read_csv('ratings_train.txt', sep='\t', quoting=3))
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [2]:
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


## 토큰화 및 품사부착

In [3]:
# 토큰화 작업

content=' '
for text in df['document']:
    content += text
print(content[:40], type(content))

 아 더빙.. 진짜 짜증나네요 목소리흠...포스터보고 초딩영화줄....오 <class 'str'>


In [4]:
from konlpy.tag import Mecab
mecab = Mecab()
pos = mecab.pos(content)
print(pos[:40])

[('아', 'IC'), ('더', 'MAG'), ('빙', 'MAG'), ('.', 'SF'), ('.', 'SY'), ('진짜', 'MAG'), ('짜증', 'NNG'), ('나', 'NP'), ('네요', 'VCP+EC'), ('목소리', 'NNG'), ('흠', 'IC'), ('.', 'SF'), ('..', 'SY'), ('포스터', 'NNP'), ('보고', 'JKB'), ('초딩', 'NNG'), ('영화', 'NNG'), ('줄', 'NNG'), ('.', 'SF'), ('...', 'SY'), ('오버', 'NNG'), ('연기', 'NNG'), ('조차', 'JX'), ('가볍', 'VA'), ('지', 'EC'), ('않', 'VX'), ('구나', 'EF'), ('너무', 'MAG'), ('재', 'XPN'), ('밓었다그래서보는것을추천한다교도소', 'UNKNOWN'), ('이야기', 'NNG'), ('구먼', 'VCP+EF'), ('.', 'SF'), ('.', 'SY'), ('솔직히', 'MAG'), ('재미', 'NNG'), ('는', 'JX'), ('없', 'VA'), ('다', 'EF'), ('.', 'SF')]


In [5]:
# 토큰화 작업, 포스 태깅

from konlpy.tag import Mecab
mecab = Mecab()
mecab_pos = []
for i, doc in enumerate(df['document']):
    mecab_pos.append(mecab.pos(doc))
mecab_pos

[[('아', 'IC'),
  ('더', 'MAG'),
  ('빙', 'MAG'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('진짜', 'MAG'),
  ('짜증', 'NNG'),
  ('나', 'NP'),
  ('네요', 'VCP+EC'),
  ('목소리', 'NNG')],
 [('흠', 'IC'),
  ('.', 'SF'),
  ('..', 'SY'),
  ('포스터', 'NNP'),
  ('보고', 'JKB'),
  ('초딩', 'NNG'),
  ('영화', 'NNG'),
  ('줄', 'NNG'),
  ('.', 'SF'),
  ('...', 'SY'),
  ('오버', 'NNG'),
  ('연기', 'NNG'),
  ('조차', 'JX'),
  ('가볍', 'VA'),
  ('지', 'EC'),
  ('않', 'VX'),
  ('구나', 'EC')],
 [('너무', 'MAG'), ('재', 'XPN'), ('밓었다그래서보는것을추천한다', 'UNKNOWN')],
 [('교도소', 'NNG'),
  ('이야기', 'NNG'),
  ('구먼', 'VCP+EF'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('솔직히', 'MAG'),
  ('재미', 'NNG'),
  ('는', 'JX'),
  ('없', 'VA'),
  ('다', 'EF'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('평점', 'NNG'),
  ('조정', 'NNG')],
 [('사이몬페그', 'NNP'),
  ('의', 'JKG'),
  ('익살', 'NNG'),
  ('스런', 'XSA+ETM'),
  ('연기', 'NNG'),
  ('가', 'JKS'),
  ('돋보였', 'VV+EP'),
  ('던', 'ETM'),
  ('영화', 'NNG'),
  ('!', 'SY'),
  ('스파이더맨', 'NNP'),
  ('에서', 'JKB'),
  ('늙', 'VV'),
  ('어', 'EC'),
  ('보이', 'VV'),
  ('기',

In [6]:
from collections import Counter
Counter(pos).most_common()

[(('.', 'SF'), 119800),
 (('영화', 'NNG'), 57288),
 (('고', 'EC'), 46301),
 (('는', 'ETM'), 45281),
 (('이', 'JKS'), 42613),
 (('.', 'SY'), 40478),
 (('의', 'JKG'), 33565),
 (('도', 'JX'), 33431),
 (('다', 'EF'), 32805),
 (('에', 'JKB'), 30670),
 (('가', 'JKS'), 28852),
 (('을', 'JKO'), 23632),
 (('..', 'SY'), 22744),
 ((',', 'SC'), 21907),
 (('는', 'JX'), 21384),
 (('들', 'XSN'), 20001),
 (('보', 'VV'), 19687),
 (('이', 'VCP'), 19406),
 (('한', 'XSA+ETM'), 18103),
 (('은', 'JX'), 17335),
 (('게', 'EC'), 17039),
 (('를', 'JKO'), 16972),
 (('하', 'XSV'), 16009),
 (('없', 'VA'), 15858),
 (('하', 'XSA'), 13927),
 (('은', 'ETM'), 13783),
 (('좋', 'VA'), 13773),
 (('!', 'SF'), 13470),
 (('하', 'VV'), 12839),
 (('?', 'SF'), 12680),
 (('지', 'EC'), 12376),
 (('었', 'EP'), 12195),
 (('다', 'EC'), 11722),
 (('너무', 'MAG'), 10924),
 (('는데', 'EC'), 9700),
 (('로', 'JKB'), 9650),
 (('정말', 'MAG'), 9642),
 (('안', 'MAG'), 9583),
 (('으로', 'JKB'), 9286),
 (('적', 'XSN'), 9263),
 (('것', 'NNB'), 9225),
 (('만', 'JX'), 9146),
 (('재밌', '

## 불용어 처리

In [7]:
# 불용어 처리
#stopPos = 'SF SE SSO SSC SC SY SH SL SN EP EF EC ETN ETM XPN JKS JKC JKG JKO JKB JKV JKQ JC JX MM MAG MAJ XSN XSV XSA XR'.split(' ')
stopPos = ['SF','EC','ETM','JKS','SY','JKG','JX','EF','JKB',
           'JKS','JKO','SC','XSN','VV','VCP','XSA+ETM','JKO',
           'XSV','XSA','SN','SL','SH','SY','SE','SSO','SSC','SC']
#stopPos = []#['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX']
stopWord = ''.split()
X_train = []
for i, content in enumerate(mecab_pos):
    clear = []
    for pos in content:
        if pos[1] not in stopPos:
            if pos[0] not in stopWord:
                #if (len(pos[0]) != 1):
                clear.append(pos[0])
    X_train.append(' '.join(clear))

In [8]:
X_train

['아 더 빙 진짜 짜증 나 네요 목소리',
 '흠 포스터 초딩 영화 줄 오버 연기 가볍 않',
 '너무 재 밓었다그래서보는것을추천한다',
 '교도소 이야기 구먼 솔직히 재미 없 평점 조정',
 '사이몬페그 익살 연기 돋보였 영화 스파이더맨 기 했 커스틴 던스트 너무나 이뻐 보였',
 '막 걸음마 뗀 세 초등 학교 학년 인 살 영화 ㅋㅋㅋ 별반 개 아까움',
 '원작 긴장감 제대로 살려 내 못했',
 '별 반개 아깝 욕 나온다 이응경 길용우 연기 생활 몇 년 인지 정말 발 해도 그것 보단 낫 겟 납치 감금 반복 반복 이 드라마 가족 없 연기 못 사람 모엿 네',
 '액션 없 재미 몇 안 영화',
 '왜 케 평점 낮 건데 꽤 볼 만한데 헐리우드 화려 함 너무 길들여져 있',
 '걍인피니트가짱이다 진짜 짱',
 '볼 때 겠 년 대 향수 자극 허진호 감성 절제 멜로 달인',
 '횡단 보도 건널 때 뛰쳐나올 뻔 이범수 연기 드럽 못해',
 '담백 깔끔 해서 좋 신문 기사 보다 자꾸 잊어버린다 그 사람 었 것',
 '취향 존중 한다지만 진짜 내생 극장 본 영화 중 가장 노 잼 노 감동 임 스토리 어 감동 어거지',
 'ㄱ 냥 매번 긴장 재 밋 음 ㅠㅠ',
 '참 사람 웃긴 게 바스코 이기면 락스 코 라고 바비 아이돌 깐다 그냥 싶 안달 것 보인다',
 '굿바이 레닌 표절 인 것 이해 왜 뒤 갈수록 재미없 지',
 '이건 정말 깨알 캐스팅 과 질퍽 않 산뜻 내용 구성 잘 러진 깨알 일 드',
 '약탈자 위한 변명 이라 저놈 착한 놈 절대 아닌걸요',
 '나름 심오 뜻 있 듯 그냥 학생 선생 영화 절대 아님',
 '않 건 불 가능',
 '재미없 지루 같 음식 영화 인데 바베트 만찬 넘 바베트 만찬 이야기 있 음식 재미 있 이건 볼 게 없 음식 별로 안 핀란드 풍경 구경 할 랫 그것 별로 안 나옴 ㅡㅡ',
 '절대 평범 영화 가 아닌 수작 걸 말씀 드립니다',
 '주제 좋 중반 지루',
 '다 짤랐 꺼 그래서 납득 할 수 없 었 거 야 그럴 꺼 꼭 그랬 걸',
 '고추 버려

In [9]:
# 한 글자 , 숫자, 영어 삭제
Y_train = list(df['label'])
print(X_train[:10])
print(Y_train[:10])

['아 더 빙 진짜 짜증 나 네요 목소리', '흠 포스터 초딩 영화 줄 오버 연기 가볍 않', '너무 재 밓었다그래서보는것을추천한다', '교도소 이야기 구먼 솔직히 재미 없 평점 조정', '사이몬페그 익살 연기 돋보였 영화 스파이더맨 기 했 커스틴 던스트 너무나 이뻐 보였', '막 걸음마 뗀 세 초등 학교 학년 인 살 영화 ㅋㅋㅋ 별반 개 아까움', '원작 긴장감 제대로 살려 내 못했', '별 반개 아깝 욕 나온다 이응경 길용우 연기 생활 몇 년 인지 정말 발 해도 그것 보단 낫 겟 납치 감금 반복 반복 이 드라마 가족 없 연기 못 사람 모엿 네', '액션 없 재미 몇 안 영화', '왜 케 평점 낮 건데 꽤 볼 만한데 헐리우드 화려 함 너무 길들여져 있']
[0, 1, 0, 0, 1, 0, 0, 0, 1, 1]


## 베이지안 모델 머신러닝

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

# CountVectorizer 선언
count_vect = CountVectorizer()

# fit and transform
X_train_counts = count_vect.fit_transform(X_train)

# MultinomialNB 선언 and fir
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [11]:
# 예측
print(clf.predict(count_vect.transform(['영화 재미 없'])))

# 확률
print(clf.predict_proba(count_vect.transform(['영화 재미 없'])))

[0]
[[0.57508861 0.42491139]]


## 테스트 데이터 호출

In [12]:
test = pd.DataFrame(pd.read_csv('ratings_test.txt', sep='\t', quoting=3))
test = test.dropna()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [13]:
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


## 테스트 데이터 정제화

In [14]:
# 토큰화 작업, 포스 태깅

from konlpy.tag import Mecab
mecab = Mecab()
mecab_pos_test = []
for i, doc in enumerate(test['document']):
    mecab_pos_test.append(mecab.pos(doc))
mecab_pos_test

[[('굳', 'NNG'), ('ㅋ', 'IC')],
 [('GDNTOPCLASSINTHECLUB', 'SL')],
 [('뭐', 'NP'),
  ('야', 'VCP+EF'),
  ('이', 'MM'),
  ('평점', 'NNG'),
  ('들', 'XSN'),
  ('은', 'JX'),
  ('.', 'SF'),
  ('...', 'SY'),
  ('나쁘', 'VA'),
  ('진', 'EC+JX'),
  ('않', 'VX'),
  ('지만', 'EC'),
  ('10', 'SN'),
  ('점', 'NNBC'),
  ('짜리', 'XSN'),
  ('는', 'JX'),
  ('더더욱', 'MAG'),
  ('아니', 'VCN'),
  ('잖아', 'EC')],
 [('지루', 'XR'),
  ('하', 'XSA'),
  ('지', 'EC'),
  ('는', 'JX'),
  ('않', 'VX'),
  ('은데', 'EC'),
  ('완전', 'NNG'),
  ('막장', 'NNG'),
  ('임', 'VCP+ETN'),
  ('.', 'SF'),
  ('..', 'SY'),
  ('돈', 'NNG'),
  ('주', 'VV'),
  ('고', 'EC'),
  ('보', 'VX'),
  ('기', 'ETN'),
  ('에', 'JKB'),
  ('는', 'JX'),
  ('.', 'SF'),
  ('...', 'SY')],
 [('3', 'SN'),
  ('D', 'SL'),
  ('만', 'JX'),
  ('아니', 'VCN'),
  ('었', 'EP'),
  ('어도', 'EC'),
  ('별', 'MM'),
  ('다섯', 'NR'),
  ('개', 'NNBC'),
  ('줬', 'VV+EP'),
  ('을', 'ETM'),
  ('텐데', 'NNB+VCP+EF'),
  ('.', 'SF'),
  ('.', 'SY'),
  ('왜', 'MAG'),
  ('3', 'SN'),
  ('D', 'SL'),
  ('로', 'JKB'),
  ('나와서', 'VV+

In [15]:
# 불용어 처리
#stopPos = 'SF SE SSO SSC SC SY SH SL SN EP EF EC ETN ETM XPN JKS JKC JKG JKO JKB JKV JKQ JC JX MM MAG MAJ XSN XSV XSA XR'.split(' ')
stopPos = ['SF','EC','ETM','JKS','SY','JKG','JX','EF','JKB',
           'JKS','JKO','SC','XSN','VV','VCP','XSA+ETM','JKO',
           'XSV','XSA','SN','SL','SH','SY','SE','SSO','SSC','SC']
#stopPos = []#['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX']
stopWord = ''.split()
X_test = []
for i, content in enumerate(mecab_pos_test):
    clear = []
    for pos in content:
        if pos[1] not in stopPos:
            if pos[0] not in stopWord:
                #if (len(pos[0]) != 1):
                clear.append(pos[0])
    X_test.append(' '.join(clear))

In [16]:
# X_test = test['document']
Y_test=[]
for text in X_test:
    result=''
    result=clf.predict(count_vect.transform([text])) 
    Y_test.append(result)

In [17]:
Y_test

[array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([1]),
 array([0]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([1]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([1]),

In [18]:
len(Y_test)

49997

In [19]:
len(X_test)

49997

## 정확도 확인

In [21]:
count = 0
for i, label in enumerate(test['label']):
    if (Y_test[i] == label):
        count += 1

In [24]:
print(round(count/len(Y_test)*100, 2), "%")

80.47 %


# 네이버 영화리뷰 감성분석 : 나이브베이지안 Using sklearn
- 다음의 코드는 강의 자료를 참고하여 작성한 것이다.

In [26]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('ratings_train.txt', sep='\t', quoting=3))
df = df.replace(np.nan, '', regex=True)
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [27]:
import re 
# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
def preprocess(text):
    text = re.sub(remove_except_ko,' ',text).strip()
    return text

df['document'] = df['document'].map(lambda x : preprocess(x))

In [28]:
df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [30]:
from konlpy.tag import Mecab
mecab = Mecab()
# mecab을 사용해서 토큰화, 한글자 제거, stop_word 제거 
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']

# def token_mecab(text):
#     text = mecab.morphs(text)
#     text = [i for i in text if len(i)>1]
#     text = [i for i in text if i not in stop_word]
#     return text

stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']
def token_mecab(text):
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])>1]
    text = [i for i in text if i[0] not in stop_word]
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [31]:
token_mecab('교도소 이야기구먼 솔직히 재미는 없다')

['교도소', '이야기', '구먼', '재미']

In [32]:
# 토큰화 + 토큰리스트 생성 
def make_tokens(df):
    df['tokens'] = ''
    tokens_list=[]
    for i, row in df.iterrows():
        if i%10000==0:
            print(i,'/',len(df))
        token = token_mecab(df['document'][i])
        df['tokens'][i] = ' '.join(token)
    return df

df = make_tokens(df)

0 / 150000


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 150000
20000 / 150000
30000 / 150000
40000 / 150000
50000 / 150000
60000 / 150000
70000 / 150000
80000 / 150000
90000 / 150000
100000 / 150000
110000 / 150000
120000 / 150000
130000 / 150000
140000 / 150000


In [33]:
df.head()

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,짜증 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,포스터 초딩 영화 오버 연기 가볍 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,교도소 이야기 구먼 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 보이 커스틴 던스트 이뻐 보였


In [34]:
X_train = list(df['tokens'])
Y_train = list(df['label'])

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
# CountVectorizer 선언
count_vect = CountVectorizer()
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [36]:
test = '강력한 몰입감을 선사하는 두 배우의 연기력!! 오랜만에 정말 색다른 영화였다'
test = token_mecab(test)
test = ' '.join(test)
print(test)
# 예측
print(clf.predict(count_vect.transform([test]))) 
# 확률 
print(clf.predict_proba(count_vect.transform([test])))

강력 입감 선사 배우 연기력 오랜만 색다른 영화
[1]
[[0.00266325 0.99733675]]


## 예측 결과 정확도 측정

In [38]:
import pandas as pd 
test_df = pd.DataFrame(pd.read_csv('ratings_test.txt', sep='\t', quoting=3))
test_df = test_df.replace(np.nan, '', regex=True)
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [39]:
test_df['document'] = test_df['document'].map(lambda x : preprocess(x))

In [40]:
test_df = make_tokens(test_df)
test_df.head()

0 / 50000


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


10000 / 50000
20000 / 50000
30000 / 50000
40000 / 50000


,id,document,label,tokens
0,6270596,굳 ㅋ,1,
1,9274899,,0,
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편


In [41]:
predic_label = []
for i, row in test_df.iterrows():
    predic = clf.predict(count_vect.transform([test_df['tokens'][i]]))[0]
    predic_label.append(predic)
test_df['predic_label'] = predic_label
test_df.head()

,id,document,label,tokens,predic_label
0,6270596,굳 ㅋ,1,,0
1,9274899,,0,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편,0


## confusion metrix
| 표   | 참 | 거짓 |
|------|----|------|
| 참   | TP | FP   |
| 거짓 | FN | TN   |

In [42]:
TP = FN = FP = TN = other = 0
for i in range(len(test_df)):
    if test_df['label'][i] == 1 and test_df['predic_label'][i] == 1:
        TP +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 0:
        TN +=1
    elif test_df['label'][i] == 0 and test_df['predic_label'][i] == 1:
        FN +=1
    elif test_df['label'][i] == 1 and test_df['predic_label'][i] == 0:
        FP +=1
    else:
        other +=1
print(TP,TN,FN,FP)

19632 20393 4434 5541


In [43]:
ACC = (TP+TN)/(TP+TN+FP+FN)
print('ACC: ',ACC)

ACC:  0.8005
